In [4]:
# Install web frameworks + utils
!pip install -q streamlit flask fastapi uvicorn pillow python-multipart

# Download cloudflared (for public URL, no account needed)
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!mv cloudflared-linux-amd64 cloudflared
!chmod +x cloudflared

**STREAMLIT**

In [5]:
%%writefile streamlit_app.py
import streamlit as st
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load model once at app start
model = load_model("currency_model.h5")
IMG_SIZE = (224, 224)

def predict(img: Image.Image):
    # preprocess
    img = img.convert("RGB")
    img = img.resize(IMG_SIZE)
    arr = np.array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)
    # predict
    pred = model.predict(arr)[0][0]
    if pred >= 0.5:
        label = "Real Currency Note"
        confidence = pred * 100
    else:
        label = "Fake Currency Note"
        confidence = (1 - pred) * 100
    return label, float(confidence)

st.title("🇵🇰 Pakistani Currency Real/Fake Detector")

uploaded = st.file_uploader("Upload a PKR note image", type=["jpg", "jpeg", "png"])
if uploaded is not None:
    img = Image.open(uploaded)
    st.image(img, caption="Uploaded note", width=350)

    label, conf = predict(img)
    st.success(f"Prediction: **{label}**")
    st.info(f"Confidence: **{conf:.2f}%**")
else:
    st.write("Please upload an image.")


Writing streamlit_app.py


In [6]:
# Run streamlit in background on port 7860
!nohup streamlit run streamlit_app.py --server.address 0.0.0.0 --server.port 7860 >/dev/null 2>&1 &

# Expose it to internet via cloudflared
!./cloudflared tunnel --url http://0.0.0.0:7860 --no-autoupdate

2025-11-30T18:43:08Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-30T18:43:08Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-30T18:43:12Z INF +--------------------------------------------------------------------------------------------+
2025-11-30T18:43:12Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-30T18:43:12Z INF |  https://goat-grain-select-relatives.trycloudflare.com

**FLASK**

In [7]:
%%writefile flask_app.py
import io
import numpy as np
from PIL import Image
import tensorflow as tf
from flask import Flask, request, render_template_string

# ---------- Model & Preprocessing ----------
IMG_SIZE = (224, 224)

model = tf.keras.models.load_model("currency_model.h5")
# index 0 = fake, 1 = real  (same as your generator)
class_names = ["Fake Currency Note", "Real Currency Note"]

def predict_image(img: Image.Image):
    img = img.convert("RGB")
    img = img.resize(IMG_SIZE)
    arr = np.array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)
    preds = model.predict(arr)[0]
    idx = int(np.argmax(preds))
    conf = float(preds[idx] * 100)
    label = class_names[idx]
    return label, conf

# ---------- Flask App ----------
app = Flask(__name__)

HTML_PAGE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8" />
    <title>PKR Currency Real/Fake Detector · Flask</title>
    <style>
        :root {
            --bg: #020617;
            --bg-soft: #020617dd;
            --card: #020617;
            --accent: #38bdf8;
            --accent-soft: #0ea5e9;
            --border: #1f2937;
            --text-main: #e5e7eb;
            --text-muted: #9ca3af;
            --danger: #fb7185;
            --success: #22c55e;
        }
        * { box-sizing: border-box; }
        body {
            margin: 0;
            font-family: system-ui, -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
            background: radial-gradient(circle at top left, #0f172a, #020617 50%, #0b1120 100%);
            color: var(--text-main);
        }
        .page {
            min-height: 100vh;
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 32px 16px;
        }
        .card {
            width: 100%;
            max-width: 960px;
            background: var(--bg-soft);
            border-radius: 24px;
            border: 1px solid var(--border);
            box-shadow: 0 24px 80px rgba(15, 23, 42, 0.9);
            padding: 32px 28px 28px;
        }
        .header {
            display: flex;
            align-items: center;
            gap: 12px;
            margin-bottom: 24px;
        }
        .logo-pill {
            width: 40px;
            height: 40px;
            border-radius: 999px;
            background: linear-gradient(135deg, #22c55e, #38bdf8);
            display: flex;
            align-items: center;
            justify-content: center;
            font-weight: 700;
            font-size: 20px;
            color: #0b1120;
        }
        h1 {
            margin: 0;
            font-size: 26px;
            letter-spacing: 0.02em;
        }
        .subtitle {
            margin-top: 2px;
            font-size: 14px;
            color: var(--text-muted);
        }
        .grid {
            display: grid;
            grid-template-columns: minmax(0, 1.4fr) minmax(0, 1fr);
            gap: 24px;
        }
        @media (max-width: 800px) {
            .grid { grid-template-columns: minmax(0,1fr); }
        }
        .upload-card, .result-card {
            background: rgba(15, 23, 42, 0.75);
            border-radius: 18px;
            border: 1px solid var(--border);
            padding: 18px 18px 20px;
        }
        .section-title {
            font-size: 15px;
            font-weight: 600;
            margin-bottom: 6px;
        }
        .section-caption {
            font-size: 13px;
            color: var(--text-muted);
            margin-bottom: 14px;
        }
        .upload-box {
            border-radius: 16px;
            border: 1px dashed rgba(148, 163, 184, 0.7);
            padding: 22px 16px;
            text-align: center;
            background: rgba(15, 23, 42, 0.7);
        }
        .upload-box:hover {
            border-color: var(--accent);
            background: rgba(15, 23, 42, 0.9);
        }
        .upload-box input[type="file"] {
            display: block;
            margin: 10px auto 6px;
            cursor: pointer;
            color: var(--text-muted);
        }
        .helper-text {
            font-size: 12px;
            color: var(--text-muted);
        }
        .btn {
            margin-top: 14px;
            display: inline-flex;
            align-items: center;
            justify-content: center;
            padding: 10px 18px;
            border-radius: 999px;
            border: none;
            font-size: 14px;
            font-weight: 500;
            background: linear-gradient(135deg, var(--accent), var(--accent-soft));
            color: #020617;
            cursor: pointer;
            box-shadow: 0 12px 30px rgba(56, 189, 248, 0.45);
        }
        .btn:hover {
            filter: brightness(1.05);
            box-shadow: 0 18px 40px rgba(56, 189, 248, 0.6);
        }
        .btn:active {
            transform: translateY(1px);
            box-shadow: 0 8px 20px rgba(56, 189, 248, 0.35);
        }
        .result-empty {
            font-size: 13px;
            color: var(--text-muted);
            margin-top: 4px;
        }
        .tag {
            display: inline-flex;
            align-items: center;
            gap: 6px;
            padding: 4px 12px;
            border-radius: 999px;
            font-size: 12px;
            font-weight: 600;
        }
        .tag-real {
            background: rgba(34, 197, 94, 0.1);
            color: #bbf7d0;
            border: 1px solid rgba(34, 197, 94, 0.7);
        }
        .tag-fake {
            background: rgba(248, 113, 113, 0.12);
            color: #fecaca;
            border: 1px solid rgba(248, 113, 113, 0.7);
        }
        .result-label {
            margin-top: 10px;
            font-size: 19px;
            font-weight: 600;
        }
        .confidence {
            margin-top: 8px;
            font-size: 14px;
        }
        .confidence span {
            font-weight: 600;
            color: var(--accent);
        }
        .footer-note {
            margin-top: 12px;
            font-size: 11px;
            color: var(--text-muted);
        }
    </style>
</head>
<body>
<div class="page">
    <div class="card">
        <div class="header">
            <div class="logo-pill">PK</div>
            <div>
                <h1>Pakistani Currency Real/Fake Detector</h1>
                <div class="subtitle">Upload a PKR note image to verify whether it looks genuine or counterfeit.</div>
            </div>
        </div>

        <div class="grid">
            <!-- Upload side -->
            <div class="upload-card">
                <div class="section-title">1 · Upload PKR note image</div>
                <div class="section-caption">Supported formats: JPG, JPEG, PNG (up to ~5MB in Colab).</div>
                <form method="POST" enctype="multipart/form-data">
                    <div class="upload-box">
                        <input type="file" name="file" accept="image/*" required />
                        <div class="helper-text">Click to choose an image from your device.</div>
                    </div>
                    <button class="btn" type="submit">Analyse Note</button>
                </form>
            </div>

            <!-- Result side -->
            <div class="result-card">
                <div class="section-title">2 · Result</div>
                {% if result %}
                    <div class="section-caption">AI model prediction based on visual features of the uploaded note.</div>
                    <div class="tag {{ label_class }}">
                        {% if 'Real' in result %}
                            <span>●</span> Real Currency
                        {% else %}
                            <span>●</span> Suspected Fake
                        {% endif %}
                    </div>
                    <div class="result-label">{{ result }}</div>
                    <div class="confidence">
                        Confidence score: <span>{{ '%.2f'|format(confidence or 0.0) }}%</span>
                    </div>
                    <div class="footer-note">
                        This is an AI-based assistance only. Always verify with official sources for legal or financial decisions.
                    </div>
                {% else %}
                    <div class="section-caption">No prediction yet.</div>
                    <div class="result-empty">
                        Upload a currency note on the left and click <b>Analyse Note</b> to see the prediction here.
                    </div>
                {% endif %}
            </div>
        </div>
    </div>
</div>
</body>
</html>
"""

@app.route("/", methods=["GET", "POST"])
def index():
    result = None
    confidence = None
    label_class = ""

    if request.method == "POST":
        if "file" in request.files:
            file = request.files["file"]
            if file and file.filename != "":
                img_bytes = file.read()
                img = Image.open(io.BytesIO(img_bytes))
                result, confidence = predict_image(img)
                # Choose badge color
                label_class = "tag-real" if "Real" in result else "tag-fake"
            else:
                result = "No selected file"
                label_class = "tag-fake"

    return render_template_string(
        HTML_PAGE,
        result=result,
        confidence=confidence,
        label_class=label_class
    )

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000, debug=False)


Writing flask_app.py


In [8]:
!nohup python flask_app.py > flask_log.txt 2>&1 &
!./cloudflared tunnel --url http://localhost:5000 --no-autoupdate

2025-11-30T19:26:08Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-30T19:26:08Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-30T19:26:12Z INF +--------------------------------------------------------------------------------------------+
2025-11-30T19:26:12Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-30T19:26:12Z INF |  https://disc-nominations-harder-auckland.trycloudflar

**FASTAPI**

In [13]:
!mkdir -p static
!mkdir -p templates

In [14]:
%%writefile static/style.css
/* Your full CSS here */

Writing static/style.css


In [15]:
%%writefile templates/index.html
<!-- Your full HTML here -->

Writing templates/index.html


In [ ]:
from fastapi import FastAPI, Request, UploadFile, File
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates

from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import io

app = FastAPI(title="PKR Note Detector - FastAPI")

# Mount static files (for CSS)
app.mount("/static", StaticFiles(directory="static"), name="static")

# Templates folder
templates = Jinja2Templates(directory="templates")

IMG_SIZE = (224, 224)

# ---------- Load model on startup ----------
@app.on_event("startup")
async def load_currency_model():
    global model
    model = load_model("currency_model.h5")
    print("✅ FastAPI: model loaded successfully")


# ---------- Helper: predict ----------
def predict_currency(image_bytes: bytes):
    img = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    img = img.resize(IMG_SIZE)
    arr = np.array(img) / 255.0
    arr = np.expand_dims(arr, axis=0)

    preds = model.predict(arr)[0]  # e.g. [prob_fake, prob_real]
    prob_fake = float(preds[0])
    prob_real = float(preds[1])

    if prob_real >= prob_fake:
        label = "Real Currency Note"
        confidence = prob_real * 100
    else:
        label = "Fake Currency Note"
        confidence = prob_fake * 100

    return label, round(confidence, 2)


# ---------- Routes ----------
@app.get("/", response_class=HTMLResponse)
async def home(request: Request):
    # First load = no prediction yet
    return templates.TemplateResponse(
        "index.html",
        {
            "request": request,
            "prediction": None,
            "confidence": None,
            "note_class": None,
        },
    )


@app.post("/predict", response_class=HTMLResponse)
async def predict(
    request: Request,
    file: UploadFile = File(...)
):
    # Read uploaded file
    image_bytes = await file.read()
    label, confidence = predict_currency(image_bytes)

    # Class for coloring chip (success / danger)
    note_class = "real" if "Real" in label else "fake"

    return templates.TemplateResponse(
        "index.html",
        {
            "request": request,
            "prediction": label,
            "confidence": confidence,
            "note_class": note_class,
            "filename": file.filename,
        },
    )


In [ ]:
!uvicorn fastapi_app:app --host 0.0.0.0 --port 7860 --reload &
!./cloudflared tunnel --url http://0.0.0.0:7860 --no-autoupdate